In [30]:
# Hacking a non-streaming CNN Modelfrom subprocess import check_output
import matplotlib.pyplot as plt
%matplotlib inline 

from skimage.data import imread   
import multiprocessing as mp      
import bson
import io
import numpy as np 
import pandas as pd 


import time
import threading
import itertools
from tqdm import *

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Lambda
from keras.optimizers import Adam
from keras.models import Sequential, load_model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical

import tensorflow as tf
import pickle

In [4]:
# Looking specifically over the range for the following categories, while aready numeric
# I will use label encoder to take out any sense of ordinal value 

# photography books, auto-fuses, headboards, hinges, lotion
targets = [1000014042, 1000005888, 1000015754, 1000002844, 1000003988]

In [5]:
label_encoder = LabelEncoder()
label_encoder.fit(targets)

LabelEncoder()

In [6]:
data = bson.decode_file_iter(open('/Volumes/G-DRIVE mobile/Image_Data_DSI/train.bson', 'rb'))

In [7]:
# load data and convert it into an iterable version of the data

In [8]:
it = iter(data)

In [9]:
# This is the method to stream through the test set and pull out the data 
X = []
y = []
counter=0
with tqdm(total=6728) as pbar:
    for i in range(6009411):
        piece = next(it)
        if piece.get('category_id') in targets:
            counter = counter + 1
            if counter % 250 == 0: 
                print(counter)
            img = piece.get('imgs')[0]
            pic = io.BytesIO(img.get('picture', None))
            im = imread(pic)
            im = im.astype('float32') / 255.0
            X.append(im)
            np.array(X)

            category = piece.get('category_id', "")
            label = label_encoder.transform([category])
            y.append(label)
            np.array(y)
            pbar.update()





  4%|▎         | 250/6728 [01:12<26:23,  4.09it/s]  

250


  7%|▋         | 500/6728 [02:21<32:58,  3.15it/s]

500


 11%|█         | 749/6728 [03:50<34:43,  2.87it/s]  

...

 93%|█████████▎| 6249/6728 [2:12:40<51:53,  6.50s/it]  

6250


 97%|█████████▋| 6499/6728 [2:30:38<11:25,  2.99s/it]  

6500


 99%|█████████▊| 6633/6728 [2:44:40<12:23,  7.83s/it]


In [31]:
len(X)


6633

In [32]:
X =np.array(X)

In [33]:
type(X)

numpy.ndarray

In [34]:
y = np.array(y)

In [43]:
len(y)

33165

In [35]:
type(y)

numpy.ndarray

In [36]:
with open('MB_Xtrain.npy', 'wb') as np_mbX_out:
    np.save(np_mbX_out,X)

In [37]:
#y = to_categorical(y)

In [44]:
y.shape

(33165, 2)

In [38]:
with open('MB_ytrain.npy', 'wb') as np_mby_out:
    np.save(np_mby_out,y)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2003)

X_train = X_train.reshape(X_train.shape[0], 180, 180, 3)
X_test = X_test.reshape(X_test.shape[0], 180, 180, 3)


In [15]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(180, 180, 3), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(16, (4,4), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2)

Train on 4974 samples, validate on 1659 samples
Epoch 1/2
4974/4974 [==============================] - 217s 44ms/step - loss: 0.9476 - acc: 0.6711 - val_loss: 0.7067 - val_acc: 0.7951
Epoch 2/2
4974/4974 [==============================] - 215s 43ms/step - loss: 0.5876 - acc: 0.8070 - val_loss: 0.5957 - val_acc: 0.8222


In [16]:
model.save("KPS_diff_Model_1")

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 176, 176, 32)      2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 88, 88, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 85, 85, 16)        8208      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 42, 42, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28224)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                903200    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 165       
Total para

In [18]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Train on 4974 samples, validate on 1659 samples
Epoch 1/3
4974/4974 [==============================] - 218s 44ms/step - loss: 0.3653 - acc: 0.8790 - val_loss: 0.6420 - val_acc: 0.8125
Epoch 2/3
4974/4974 [==============================] - 209s 42ms/step - loss: 0.2428 - acc: 0.9176 - val_loss: 0.6691 - val_acc: 0.8342
Epoch 3/3
4974/4974 [==============================] - 210s 42ms/step - loss: 0.1369 - acc: 0.9564 - val_loss: 0.7732 - val_acc: 0.8385


In [19]:
model.save("KPS_diff_Model_1_5epochs")

In [20]:
# Now will generate true out of sample data from the remaining images in the set, will
# loop through again, since there are 7898 images in the set, know there are 1,298 unseen images
# in teh remaining ~1mm images. I will re-run the loop above, since it is an iterator, it will
# pick up on data where left off before. 

In [18]:
# This is the method to stream through the test set and pull out the data 
Xt = []
yt = []
counter=0
with tqdm(total=1283) as pbar:
    for i in range(1060480):
        piece = next(it)
        if piece.get('category_id') in targets:
            counter = counter + 1
            if counter % 500 == 0: 
                print(counter)
            img = piece.get('imgs')[0]
            pic = io.BytesIO(img.get('picture', None))
            im = imread(pic)
            im = im.astype('float32') / 255.0
            Xt.append(im)
            np.array(Xt)

            category = piece.get('category_id', "")
            label = label_encoder.transform([category])
            yt.append(label)
            np.array(yt)
            pbar.update()



 39%|███▉      | 499/1283 [02:11<03:45,  3.48it/s]

500


 78%|███████▊  | 999/1283 [05:16<01:50,  2.56it/s]

1000


100%|██████████| 1283/1283 [07:16<00:00,  2.19it/s]


In [42]:
len(y)

33165

In [40]:
len(yt) + len(y)

34448

In [41]:
len(yt)

1283

In [20]:
yt = np.array(yt)

In [21]:
#yt = to_categorical(yt)

In [24]:
# Good spot check, y and T are seperate!
# left five spaces on end of next(it) to verify was at end. So now the full dataset for 
# images I want to classify saved as variables 

In [25]:
type(Xt)

list

In [26]:
Xt = np.array(Xt)

In [27]:
with open('MB_Xtest.npy', 'wb') as np_mbXt_out:
    np.save(np_mbXt_out, Xt)

In [29]:
with open('MB_ytest.npy', 'wb') as np_mbyt_out:
    np.save(np_mbyt_out, yt)

In [30]:
preds= model.predict(Xt,32)

In [31]:
preds

array([[  3.01999826e-04,   3.62010795e-08,   9.87386167e-01,
          1.22194225e-02,   9.22527761e-05],
       [  3.58475864e-01,   2.70820572e-04,   4.68214303e-02,
          1.07964955e-01,   4.86466944e-01],
       [  5.86078912e-02,   1.27363382e-04,   9.03451025e-01,
          2.44561136e-02,   1.33576673e-02],
       ..., 
       [  2.52697532e-06,   3.28857097e-09,   2.27253771e-08,
          9.99997497e-01,   2.98068379e-08],
       [  1.17816293e-04,   3.58062935e-05,   4.66740785e-05,
          8.03596806e-03,   9.91763711e-01],
       [  9.95701492e-01,   4.28461902e-07,   2.44033663e-03,
          3.74378898e-04,   1.48338347e-03]], dtype=float32)

In [32]:
#confusion_matrix(yt, preds)

In [33]:
model.fit(X, y, validation_data=(Xt, yt), epochs=5)

Train on 6633 samples, validate on 1283 samples
Epoch 1/5
6633/6633 [==============================] - 276s 42ms/step - loss: 0.3188 - acc: 0.9166 - val_loss: 0.6329 - val_acc: 0.8472
Epoch 2/5
6633/6633 [==============================] - 264s 40ms/step - loss: 0.1336 - acc: 0.9597 - val_loss: 0.7048 - val_acc: 0.8426
Epoch 3/5
6633/6633 [==============================] - 5834s 880ms/step - loss: 0.0681 - acc: 0.9807 - val_loss: 0.7534 - val_acc: 0.8574
Epoch 4/5
6633/6633 [==============================] - 885s 133ms/step - loss: 0.0380 - acc: 0.9902 - val_loss: 0.9029 - val_acc: 0.8574
Epoch 5/5
6633/6633 [==============================] - 858s 129ms/step - loss: 0.0431 - acc: 0.9882 - val_loss: 1.1186 - val_acc: 0.8519


In [ ]:
# Export the prediction classes? 


In [34]:
# Building a Second Model

model2 = Sequential()
model2.add(Conv2D(16, (2, 2), input_shape=(180, 180, 3), activation='relu'))
model2.add(MaxPool2D((2,2)))
model2.add(Conv2D(32, (4,4), activation='relu'))
model2.add(MaxPool2D((2, 2)))
model2.add(Conv2D(64, (4,4), activation='relu'))
model2.add(MaxPool2D((2, 2)))
model2.add(Flatten())
model2.add(Dense(32, activation='relu'))
model2.add(Dense(y.shape[1], activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Train on 4974 samples, validate on 1659 samples
Epoch 1/3
4974/4974 [==============================] - 153s 31ms/step - loss: 0.9046 - acc: 0.6799 - val_loss: 0.7160 - val_acc: 0.7860
Epoch 2/3
4974/4974 [==============================] - 142s 29ms/step - loss: 0.5912 - acc: 0.8044 - val_loss: 0.5693 - val_acc: 0.8294
Epoch 3/3
4974/4974 [==============================] - 142s 28ms/step - loss: 0.4277 - acc: 0.8552 - val_loss: 0.5968 - val_acc: 0.8035


In [35]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 179, 179, 16)      208       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 89, 89, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 86, 86, 32)        8224      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 43, 43, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 40, 40, 64)        32832     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 25600)             0         
__________

In [38]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2)

Train on 4974 samples, validate on 1659 samples
Epoch 1/2
4974/4974 [==============================] - 147s 29ms/step - loss: 0.2693 - acc: 0.9115 - val_loss: 0.4931 - val_acc: 0.8608
Epoch 2/2
4974/4974 [==============================] - 142s 28ms/step - loss: 0.1706 - acc: 0.9413 - val_loss: 0.4705 - val_acc: 0.8698


In [39]:
model2.save("Model2_diff_5_epochs")

In [40]:
model2.fit(X, y, validation_data=(Xt, yt), epochs=5)

Train on 6633 samples, validate on 1283 samples
Epoch 1/5
6633/6633 [==============================] - 185s 28ms/step - loss: 0.2282 - acc: 0.9317 - val_loss: 0.5094 - val_acc: 0.8613
Epoch 2/5
6633/6633 [==============================] - 181s 27ms/step - loss: 0.1297 - acc: 0.9614 - val_loss: 0.5506 - val_acc: 0.8792
Epoch 3/5
6633/6633 [==============================] - 180s 27ms/step - loss: 0.0786 - acc: 0.9783 - val_loss: 0.6175 - val_acc: 0.8862
Epoch 4/5
6633/6633 [==============================] - 1824s 275ms/step - loss: 0.0462 - acc: 0.9858 - val_loss: 0.8877 - val_acc: 0.8504
Epoch 5/5
6633/6633 [==============================] - 185s 28ms/step - loss: 0.0638 - acc: 0.9803 - val_loss: 0.6877 - val_acc: 0.8644
